In [1]:
import logging
log_level = 25
logging.basicConfig(level=log_level) # root logger

import re
import pandas as pd

import sys
sys.path.append('E:/Code/chat/gdpr')


In [4]:
import fitz

import os
path = 'E:/Code/chat/gdpr/pdf/guidelines'
file_name = 'wp260rev01_en.pdf'
paper_name = "Guidelines on transparency under Regulation 2016/679"
full_path = os.path.join(path, file_name)
doc = fitz.open(full_path)

output_file = "../tmp/transparency.md"

In [7]:
import importlib
import src.extract_from_pdf
importlib.reload(src.extract_from_pdf)
from src.extract_from_pdf import output_doc_as_text

lines_to_delete = [] 
characters_to_replace = []
characters_to_replace.append(['“', '"'])
characters_to_replace.append(['”', '"'])
characters_to_replace.append(['‘', "'"])
characters_to_replace.append(['’', "'"])
characters_to_replace.append(['', "-"])
characters_to_replace.append(['–', "-"])




document_text = output_doc_as_text(doc, start_page = 3, end_page = 0, header_size=70, footer_size=70, lines_to_delete = lines_to_delete, characters_to_replace = characters_to_replace)

with open(output_file, 'w', encoding='utf-8') as f:
    f.write(document_text)

## Once you have the markdown file

In [2]:
import re
import pandas as pd
file_path = "../../original/transparency.md"
with open(file_path, 'r', encoding = "utf-8") as file:
    text = file.read()

lines = text.split('\n')
# get rid of empty lines
lines = [line for line in lines if line]

doc_as_array = []
notes_as_array = []
#footnote_pattern = re.compile(r'^(\[\^\d{1,2}\]:)(.*)$')
footnote_pattern = re.compile(r'^\[\^(\d{1,2})\]:(.*)$')
for entry in lines:
    footnote_match = footnote_pattern.match(entry)
    if footnote_match:
        notes_as_array.append([footnote_match.group(1), footnote_match.group(2).strip()])
    else:
        doc_as_array.append(entry)

columns = ["note_number", "text"]
df_notes = pd.DataFrame(notes_as_array, columns = columns)
df_notes = df_notes[df_notes["text"].str.strip() != '']

In [3]:
#columns = ["section", "subsection", "point", "subpoint", "heading", "text", "section_reference"]
columns = ["section", "subsection", "point", "heading", "text", "section_reference"]
section = ""
subsection = ""
point = ""
heading = False
text = ""
section_reference = ""

section_pattern = re.compile(r'([A-Z])\.\s*(.*)')
subsection_pattern = re.compile(r'([a-z])\.\s*(.*)') 
# subpoint_pattern = re.compile(r'^(\d+)\.\s+(.+)$') 
annex_pattern = re.compile(r'(?i)^ANNEX (\d+) - (.+)$') # ignore capitalization


annex_started = False
data = []
table_data = []
for entry in doc_as_array:
    if entry.strip() != '':
        section_match = section_pattern.match(entry)
        subsection_match = subsection_pattern.match(entry)
        annex_match = annex_pattern.match(entry)

        if section_match:
            if annex_started:
                data.append(["", "", "", False, entry, "Annex"])
            else:
                match = section_match
                section = match.group(1)
                subsection = ""
                point = ""
                heading = True
                text = match.group(2)
                section_reference = section
                data.append([section, subsection, point, heading, text, section_reference])
        elif subsection_match:
            match = subsection_match
            section = section
            subsection = match.group(1)
            point = ""
            heading = True
            text = match.group(2)
            section_reference = section + "." + subsection
            data.append([section, subsection, point, heading, text, section_reference])
        elif annex_match:
            match = annex_match
            annex_started = True
            section = "Annex"
            subsection = ""
            point = ""
            heading = True
            text = match.group(1)
            section_reference = section
            data.append([section, subsection, point, heading, text, section_reference])
        
        else:
            if annex_started:
                data.append(["", "", "", False, entry, "Annex"])
            else:
                section = section
                subsection = subsection
                point = point
                heading = False
                text = entry
                section_reference = section_reference
                
                data.append(["", "", "", heading, text, section_reference])




df = pd.DataFrame(data, columns = columns)

#df.loc[df["section_reference"] == "",  "section_reference"] = "INTRODUCTION"
# Remove my note about the table
#df = df[df["text"] != 'Note this table contains a column "References to BCR-C, application form BCR-C, and / or supporting documents[^14]" which is empty in the document because it is supposed to be filled out by the controller'] 

In [4]:
df[160:180]
#df.iloc[192]["text"]
#df[df["text"] == " "]

,section,subsection,point,heading,text,section_reference
160,,,,False,Bank A is subject to a mandatory requirement u...,H.g
161,H,h,,True,Obtaining or disclosing is expressly laid down...,H.h
162,,,,False,66. Article 14.5(c) allows for a lifting of th...,H.h
163,,,,False,Example,H.h
164,,,,False,A tax authority is subject to a mandatory requ...,H.h
165,,,,False,from the data subjects and therefore the tax a...,H.h
166,H,i,,True,Confidentiality by virtue of a secrecy obligat...,H.i
167,,,,False,67. Article 14.5(d) provides for an exemption ...,H.i
168,,,,False,Example,H.i
169,,,,False,A medical practitioner (data controller) is un...,H.i


In [5]:
# Add footnotes
import re

def find_footnote_references(text):
    pattern = r'\[\^(\d+)\]'
    return re.findall(pattern, text)

for index, row in df.iterrows():
    footnotes = find_footnote_references(row['text'])
    if footnotes:
        augmented_note = row['text']
        for note in footnotes:
            augmented_note += f"\n[^{note}]: {df_notes[df_notes['note_number'] == note].iloc[0]['text']}"
        print(f"Row {index} augmented with footnotes")
        #print(augmented_note)
        df.at[index, "text"] = augmented_note
        #print

Row 1 augmented with footnotes
Row 2 augmented with footnotes
Row 4 augmented with footnotes
Row 28 augmented with footnotes
Row 34 augmented with footnotes
Row 39 augmented with footnotes
Row 41 augmented with footnotes
Row 42 augmented with footnotes
Row 43 augmented with footnotes
Row 45 augmented with footnotes
Row 46 augmented with footnotes
Row 47 augmented with footnotes
Row 52 augmented with footnotes
Row 60 augmented with footnotes
Row 70 augmented with footnotes
Row 71 augmented with footnotes
Row 84 augmented with footnotes
Row 100 augmented with footnotes
Row 102 augmented with footnotes
Row 104 augmented with footnotes
Row 106 augmented with footnotes
Row 108 augmented with footnotes
Row 111 augmented with footnotes
Row 115 augmented with footnotes
Row 116 augmented with footnotes
Row 118 augmented with footnotes
Row 120 augmented with footnotes
Row 121 augmented with footnotes
Row 129 augmented with footnotes
Row 146 augmented with footnotes
Row 171 augmented with footnot

In [6]:
file = "../../inputs/documents/transparency.parquet" # use parquet to deal with the complex text so I don't need to worry about escape characters
df.to_parquet(file, engine = 'pyarrow')

#df_no_table.to_csv(file, encoding = "utf-8", sep="|", index = False, na_rep="", quotechar='"')


## Check that the document class works as expected

In [9]:
import sys
sys.path.append('E:/Code/chat/gdpr')

import importlib
import gdpr_rag.documents.transparency
importlib.reload(gdpr_rag.documents.transparency)
from gdpr_rag.documents.transparency import Transparency

path_to_manual_as_csv_file = "../../inputs/documents/transparency.parquet"

doc = Transparency(path_to_manual_as_csv_file)


In [10]:
from IPython.display import Markdown, display

section = "A"
section = "C.b"
# section = "VII.B"

# section = "INTRODUCTION"
# section = "I.B.3"
# section = "2.1.3"
# section = "3.3"

# section = "1.1"


print(doc.get_heading(section))
display(Markdown(doc.get_text(section)))


C Elements of transparency under the GDPR
C.b "Clear and plain language"


# C Elements of transparency under the GDPR

## C.b "Clear and plain language"

12. With written information (and where written information is delivered orally, or by audio/ audiovisual methods, including for vision-impaired data subjects), best practices for clear writing should be followed.[^11] A similar language requirement (for "plain, intelligible language") has previously been used by the EU legislator[^12] and is also explicitly referred to in the context of consent in Recital 42 of the GDPR[^13]. The requirement for clear and plain language means that information should be provided in as simple a manner as possible, avoiding complex sentence and language structures. The information should be concrete and definitive; it should not be phrased in abstract or ambivalent terms or leave room for different interpretations. In particular the purposes of, and legal basis for, processing the personal data should be clear.

Poor Practice Examples

The following phrases are not sufficiently clear as to the purposes of processing:

- "We may use your personal data to develop new services" (as it is unclear what the "services" are or how the data will help develop them);

- "We may use your personal data for research purposes (as it is unclear what kind of "research" this refers to); and

- "We may use your personal data to offer personalised services" (as it is unclear what the "personalisation" entails).

Good Practice Examples[^14]:

- "We will retain your shopping history and use details of the products you have previously purchased to make suggestions to you for other products which we believe you will also be interested in " (it is clear that what types of data will be processed, that the data subject will be subject to targeted advertisements for products and that their data will be used to enable this);

- "We will retain and evaluate information on  your recent visits to our website and how you move around different sections of our website for analytics purposes to understand how people use our website so that we can make it more intuitive" (it is clear what type of data will be processed and the type of analysis which the controller is going to undertake); and

- "We will keep a record of the articles on our website that you have clicked on and use that information to target advertising on this website to you that is relevant to your interests, which we have identified based on articles you have read" (it is clear what the personalisation entails and how the interests attributed to the data subject have been identified).

13. Language qualifiers such as "may", "might", "some", "often" and "possible" should also be avoided. Where data controllers opt to use indefinite language, they should be able, in accordance with the principle of accountability, to demonstrate why the use of such language could not be avoided and how it does not undermine the fairness of processing. Paragraphs and sentences should be well structured, utilising bullets and indents to signal hierarchical

relationships. Writing should be in the active instead of the passive form and excess nouns should be avoided. The information provided to a data subject should not contain overly legalistic, technical or specialist language or terminology. Where the information is translated into one or more other languages, the data controller should ensure that all the translations are accurate and that the phraseology and syntax makes sense in the second language(s) so that the translated text does not have to be deciphered or re-interpreted. (A translation in one or more other languages should be provided where the controller targets[^15] data subjects speaking those languages.)

  
[^11]: See How to Write Clearly by the European Commission (2011), to be found at:  
[^12]: Article 5 of Council Directive 93/13/EEC of 5 April 1993 on unfair terms in consumer contracts  
[^13]: Recital 42 states that a declaration of consent pre-formulated by a data controller should be provided in an intelligible and easily accessible form, using clear and plain language and it should not contain unfair terms.  
[^14]: The requirement for transparency exists entirely independently of the requirement upon data controllers to ensure that there is an appropriate legal basis for the processing under Article 6.  
[^15]: For example, where the controller operates a website in the language in question and/or offers specific country options and/or facilitates the payment for goods or services in the currency of a particular member state then these may be indicative of a data controller targeting data subjects of a particular member state.